In [28]:
import json, collections, time
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="location_converter")

# Read the JSON file
with open('raw.json', 'r') as f:
    data = json.load(f)

In [39]:
with open('../locations.json', 'r') as f:
    location_cache = json.load(f)


def get_coord(location):
    if location_cache.get(location) is not None:
        return location_cache[location]

    print(f"API Call! for {location}")
    geolocator = Nominatim(user_agent="location_converter")
    time.sleep(1)

    loc = geolocator.geocode(location)
    assert loc.latitude, "Location not found"
    assert loc.longitude, "Location not found"
    
    location_cache[location] = [loc.latitude, loc.longitude, loc.address]
    return location_cache[location]

location_coordinates = {}

for i in data:
    location = i['job_posting_location']
    #####################
    if "remote" in location.lower():
        location_coordinates[location] = [["remote", "remote", "remote"]]
        continue
    #####################
    split_by = " and "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            if "more" in loc:
                continue
            coord = get_coord(location)
            location_coordinates[location].append(coord)
        continue
    #####################
    split_by = " | "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            location_coordinates[location].append(coord)
        continue
    split_by = ";"
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            print(location)
            # coord = get_coord(loc)
            # location_coordinates[location].append(coord)
        # continue

with open('../locations.json', 'w') as f:
    location_coordinates.update(location_cache)
    json.dump(location_coordinates, f, indent=2)

Research Triangle Park, North Carolina, United States; New York, New York, United States; San Jose,
Research Triangle Park, North Carolina, United States; New York, New York, United States; San Jose,
Research Triangle Park, North Carolina, United States; New York, New York, United States; San Jose,
San Jose, California, United States; Cranberry Township, Pennsylvania, United States; Boulder, Color
San Jose, California, United States; Cranberry Township, Pennsylvania, United States; Boulder, Color
San Jose, California, United States; Cranberry Township, Pennsylvania, United States; Boulder, Color
Boston; Raleigh
Boston; Raleigh
San Jose;San Francisco
San Jose;San Francisco
San Jose;San Francisco
San Jose;San Francisco
Florida - Deerfield Beach; Missouri - St. Louis
Florida - Deerfield Beach; Missouri - St. Louis
New York; Massachusetts
New York; Massachusetts
Plano, TX; Purchase, NY
Plano, TX; Purchase, NY
New York, New York, USA; Boston, Massachusetts, USA
New York, New York, USA; Bost

In [36]:

with open('../locations.json', 'w') as f:
    location_coordinates.update(location_cache)
    json.dump(location_coordinates, f, indent=2)

In [23]:
loc = geolocator.geocode("New York, NY")
from rich import inspect
inspect(loc)

╭───────────────────────────────── <class 'geopy.location.Location'> ─────────────────────────────────╮
│ Contains a parsed geocoder response. Can be iterated over as                                        │
│ ``(location<String>, (latitude<float>, longitude<Float))``.                                         │
│ Or one can access the properties ``address``, ``latitude``,                                         │
│ ``longitude``, or ``raw``. The last                                                                 │
│ is a dictionary of the geocoder's response for this item.                                           │
│                                                                                                     │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Location(City of New York, New York, United States, (40.7127281, -74.0060152, 0.0))             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                     │
│   address = 'City of New York, New York, United States'                                             │
│  altitude = 0.0                                                                                     │
│  latitude = 40.7127281                                                                              │
│ longitude = -74.0060152                                                                             │
│     point = Point(40.7127281, -74.0060152, 0.0)                                                     │
│       raw = {                                                                                       │
│                 'place_id': 332900863,                                                              │
│                 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', │
│                 'osm_type': 'relation',                                                             │
│                 'osm_id': 175905,                                                                   │
│                 'lat': '40.7127281',                                                                │
│                 'lon': '-74.0060152',                                                               │
│                 'class': 'boundary',                                                                │
│                 'type': 'administrative',                                                           │
│                 'place_rank': 10,                                                                   │
│                 'importance': 0.8716708671039277,                                                   │
│                 'addresstype': 'city',                                                              │
│                 'name': 'City of New York',                                                         │
│                 'display_name': 'City of New York, New York, United States',                        │
│                 'boundingbox': ['40.4765780', '40.9176300', '-74.2588430', '-73.7002330']           │
│             }                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────╯